In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, split
from pyspark import SparkContext
from pyspark.sql.functions import regexp_replace


creamos sesion para usar pyspark


In [2]:
SpSesion = SparkSession \
          .builder \
          .appName('etl') \
          .getOrCreate() 

Creamos la sesion de contexto

In [3]:
SpContext = SpSesion.sparkContext

vamos a cargar datos


In [4]:
path = r'C:\Users\Faccu E\Downloads\PI01\FastAPI\Data\output_steam_games.json'

In [5]:
dataset = SpSesion.read.json(path)

Vamos a pasar a explorar el archivo y ver de que se trata, tipos de columnas variables  etc


In [6]:
print('los tipos de datos son para cada columna \n',dataset.dtypes)
print()
print(f'tiene {len(dataset.columns)} columnas ')
print('las columnas son \n',dataset.columns)
print()
print(f'el dataset tiene {dataset.count()} filas')
print()



los tipos de datos son para cada columna 
 [('app_name', 'string'), ('developer', 'string'), ('early_access', 'string'), ('genres', 'string'), ('id', 'string'), ('price', 'string'), ('publisher', 'string'), ('release_date', 'string'), ('reviews_url', 'string'), ('specs', 'string'), ('tags', 'string'), ('title', 'string'), ('url', 'string')]

tiene 13 columnas 
las columnas son 
 ['app_name', 'developer', 'early_access', 'genres', 'id', 'price', 'publisher', 'release_date', 'reviews_url', 'specs', 'tags', 'title', 'url']

el dataset tiene 120445 filas



In [7]:
dataset.printSchema()

root
 |-- app_name: string (nullable = true)
 |-- developer: string (nullable = true)
 |-- early_access: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- id: string (nullable = true)
 |-- price: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- reviews_url: string (nullable = true)
 |-- specs: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)



Vamos a verlo impreso en pantalla

In [8]:
print(dataset.show())

+--------+---------+------------+------+-----+-----+---------+------------+-----------+-----+-----+-----+-----+
|app_name|developer|early_access|genres|   id|price|publisher|release_date|reviews_url|specs| tags|title|  url|
+--------+---------+------------+------+-----+-----+---------+------------+-----------+-----+-----+-----+-----+
|   "NaN"|    "NaN"|       "NaN"| "NaN"|"NaN"|"NaN"|    "NaN"|       "NaN"|      "NaN"|"NaN"|"NaN"|"NaN"|"NaN"|
|   "NaN"|    "NaN"|       "NaN"| "NaN"|"NaN"|"NaN"|    "NaN"|       "NaN"|      "NaN"|"NaN"|"NaN"|"NaN"|"NaN"|
|   "NaN"|    "NaN"|       "NaN"| "NaN"|"NaN"|"NaN"|    "NaN"|       "NaN"|      "NaN"|"NaN"|"NaN"|"NaN"|"NaN"|
|   "NaN"|    "NaN"|       "NaN"| "NaN"|"NaN"|"NaN"|    "NaN"|       "NaN"|      "NaN"|"NaN"|"NaN"|"NaN"|"NaN"|
|   "NaN"|    "NaN"|       "NaN"| "NaN"|"NaN"|"NaN"|    "NaN"|       "NaN"|      "NaN"|"NaN"|"NaN"|"NaN"|"NaN"|
|   "NaN"|    "NaN"|       "NaN"| "NaN"|"NaN"|"NaN"|    "NaN"|       "NaN"|      "NaN"|"NaN"|"NaN"|"NaN"

Contamos la cantidad de filas que tiene la tabla

In [9]:
cantidad_filas = dataset.count()

print (cantidad_filas)

120445


vamos a limpiar las filas que tengan "NaN" en la columna id dado que no queremos la información de filas sin id, dado que la columna id es la mas relevante a mi parecer dentro del dataset.

In [10]:
nan_count = dataset.filter(col('id') == '"NaN"').count()

print (nan_count)

88312


In [11]:
dataset = dataset.filter(col('id') != '"NaN"')

In [12]:
dataset.show(truncate=False)

+------------------------------------------------------------------------------+------------------------------+------------+-------------------------------------------------------+------+------------+------------------------------+------------+-------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------

Eliminamos las columnas que no necesitamos

In [13]:
dataset = dataset.drop('price')
dataset = dataset.drop('early_access')
dataset = dataset.drop('reviews_url')
dataset = dataset.drop('url')
dataset = dataset.drop('specs')
dataset = dataset.drop('tags')
dataset = dataset.drop('app_name')
dataset = dataset.drop('publisher')

In [14]:
dataset.show()

+--------------------+--------------------+------+------------+--------------------+
|           developer|              genres|    id|release_date|               title|
+--------------------+--------------------+------+------------+--------------------+
|           Kotoshiro|["Action","Casual...|761140|  2018-01-04| Lost Summoner Kitty|
|    Secret Level SRL|["Free to Play","...|643980|  2018-01-04|           Ironbound|
|        Poolians.com|["Casual","Free t...|670290|  2017-07-24|Real Pool 3D - Po...|
|            彼岸领域|["Action","Advent...|767400|  2017-12-07|          弹炸人2222|
|               "NaN"|               "NaN"|773570|       "NaN"|               "NaN"|
| Trickjump Games Ltd|["Action","Advent...|772540|  2018-01-04|Battle Royale Tra...|
|Poppermost Produc...|["Free to Play","...|774276|  2018-01-04|SNOW - All Access...|
|Poppermost Produc...|["Free to Play","...|774277|  2018-01-04|SNOW - All Access...|
|Poppermost Produc...|["Free to Play","...|774278|  2018-01-04|SNOW - Al

eliminamos las filas que no tienen informacion en la columna title

In [15]:
nan_count = dataset.filter(col('title') != '"NaN"').count()
print(nan_count)

30084


In [16]:
dataset = dataset.filter(col('title') != '"NaN"')
print(nan_count)

30084


corroboramos y eliminamos las filas que no tengan informacion en la columna genres

In [17]:
nan_count = dataset.filter(col('genres') == '"NaN"').count()
print(nan_count)

1234


In [18]:
dataset = dataset.filter(col('genres') != '"NaN"')
print(nan_count)

1234


cambiamos el nombre del DF para mas comodidad y corroboramos los cambios anteriores

In [19]:
df = dataset

In [20]:
df.show(truncate=False)

+------------------------------+-------------------------------------------------------+------+------------+------------------------------------------------------------------------------+
|developer                     |genres                                                 |id    |release_date|title                                                                         |
+------------------------------+-------------------------------------------------------+------+------------+------------------------------------------------------------------------------+
|Kotoshiro                     |["Action","Casual","Indie","Simulation","Strategy"]    |761140|2018-01-04  |Lost Summoner Kitty                                                           |
|Secret Level SRL              |["Free to Play","Indie","RPG","Strategy"]              |643980|2018-01-04  |Ironbound                                                                     |
|Poolians.com                  |["Casual","Free to Play","In

pasamos nuestro DF a pandas


In [21]:
df = dataset.toPandas()

In [22]:
df

,developer,genres,id,release_date,title
0,Kotoshiro,"[""Action"",""Casual"",""Indie"",""Simulation"",""Strat...",761140,2018-01-04,Lost Summoner Kitty
1,Secret Level SRL,"[""Free to Play"",""Indie"",""RPG"",""Strategy""]",643980,2018-01-04,Ironbound
2,Poolians.com,"[""Casual"",""Free to Play"",""Indie"",""Simulation"",...",670290,2017-07-24,Real Pool 3D - Poolians
3,彼岸领域,"[""Action"",""Adventure"",""Casual""]",767400,2017-12-07,弹炸人2222
4,Trickjump Games Ltd,"[""Action"",""Adventure"",""Simulation""]",772540,2018-01-04,Battle Royale Trainer
...,...,...,...,...,...
28845,Bidoniera Games,"[""Action"",""Adventure"",""Casual"",""Indie""]",745400,2018-01-04,Kebab it Up!
28846,"Nikita ""Ghost_RUS""","[""Casual"",""Indie"",""Simulation"",""Strategy""]",773640,2018-01-04,Colony On Mars
28847,Sacada,"[""Casual"",""Indie"",""Strategy""]",733530,2018-01-04,LOGistICAL: South Africa
28848,Laush Dmitriy Sergeevich,"[""Indie"",""Racing"",""Simulation""]",610660,2018-01-04,Russian Roads


Eliminamos los caracteres [] y "" de la columna genres

In [23]:
df['genres'] = df['genres'].str.strip('[]')

   
df['genres'] = df['genres'].str.replace('"', '')

In [24]:
df

,developer,genres,id,release_date,title
0,Kotoshiro,"Action,Casual,Indie,Simulation,Strategy",761140,2018-01-04,Lost Summoner Kitty
1,Secret Level SRL,"Free to Play,Indie,RPG,Strategy",643980,2018-01-04,Ironbound
2,Poolians.com,"Casual,Free to Play,Indie,Simulation,Sports",670290,2017-07-24,Real Pool 3D - Poolians
3,彼岸领域,"Action,Adventure,Casual",767400,2017-12-07,弹炸人2222
4,Trickjump Games Ltd,"Action,Adventure,Simulation",772540,2018-01-04,Battle Royale Trainer
...,...,...,...,...,...
28845,Bidoniera Games,"Action,Adventure,Casual,Indie",745400,2018-01-04,Kebab it Up!
28846,"Nikita ""Ghost_RUS""","Casual,Indie,Simulation,Strategy",773640,2018-01-04,Colony On Mars
28847,Sacada,"Casual,Indie,Strategy",733530,2018-01-04,LOGistICAL: South Africa
28848,Laush Dmitriy Sergeevich,"Indie,Racing,Simulation",610660,2018-01-04,Russian Roads


separamos los generos de los juegos en variables dummies para poder trabajar mas comodos en el futuro

In [25]:
def crear_columnas(df):
    dummies = df['genres'].str.get_dummies(sep=',')
    df = pd.concat([df, dummies], axis=1)
    return df

In [26]:
df=crear_columnas(df)

In [27]:
df.describe()

,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,Free to Play,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
count,28850.000000,28850.00000,28850.000000,28850.000000,28850.000000,28850.000000,28850.000000,28850.000000,28850.000000,28850.000000,...,28850.000000,28850.000000,28850.000000,28850.000000,28850.00000,28850.00000,28850.000000,28850.000000,28850.000000,28850.000000
mean,0.000243,0.39234,0.285685,0.006343,0.003224,0.287071,0.015945,0.050676,0.004333,0.070399,...,0.002669,0.189913,0.037539,0.232201,0.00364,0.04357,0.241144,0.011785,0.004021,0.009289
std,0.015575,0.48828,0.451748,0.079392,0.056686,0.452403,0.125263,0.219339,0.065682,0.255822,...,0.051594,0.392239,0.190082,0.422244,0.06022,0.20414,0.427785,0.107919,0.063283,0.095935
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.00000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.00000,1.00000,1.000000,1.000000,1.000000,1.000000


Chequeamos la cantidad de veces que se repite cada genero

In [28]:
count_ones = df.sum()
count_ones

developer                    KotoshiroSecret Level SRLPoolians.com彼岸领域Trick...
genres                       Action,Casual,Indie,Simulation,StrategyFree to...
id                           7611406439806702907674007725407742767742777742...
release_date                 2018-01-042018-01-042017-07-242017-12-072018-0...
title                        Lost Summoner KittyIronboundReal Pool 3D - Poo...
Accounting                                                                   7
Action                                                                   11319
Adventure                                                                 8242
Animation &amp; Modeling                                                   183
Audio Production                                                            93
Casual                                                                    8282
Design &amp; Illustration                                                  460
Early Access                                        

Eliminamos los generos que no consideramos escenciales para nuestro trabajo


In [29]:
df = df.drop('Photo Editing',axis=1)
df = df.drop('Video Production',axis=1)
df = df.drop('Web Publishing',axis=1)
df = df.drop('Utilities',axis=1)
df = df.drop('Early Access',axis=1)
df = df.drop('Design &amp; Illustration',axis=1)
df = df.drop('Software Training',axis=1)
df = df.drop('Free to Play',axis=1)
df = df.drop('Massively Multiplayer',axis=1)
df = df.drop('Audio Production',axis=1)
df = df.drop('genres',axis=1)
df = df.drop('Accounting',axis=1)
df = df.drop('Animation &amp; Modeling',axis=1)


In [30]:
df

,developer,id,release_date,title,Action,Adventure,Casual,Education,Indie,RPG,Racing,Simulation,Sports,Strategy
0,Kotoshiro,761140,2018-01-04,Lost Summoner Kitty,1,0,1,0,1,0,0,1,0,1
1,Secret Level SRL,643980,2018-01-04,Ironbound,0,0,0,0,1,1,0,0,0,1
2,Poolians.com,670290,2017-07-24,Real Pool 3D - Poolians,0,0,1,0,1,0,0,1,1,0
3,彼岸领域,767400,2017-12-07,弹炸人2222,1,1,1,0,0,0,0,0,0,0
4,Trickjump Games Ltd,772540,2018-01-04,Battle Royale Trainer,1,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28845,Bidoniera Games,745400,2018-01-04,Kebab it Up!,1,1,1,0,1,0,0,0,0,0
28846,"Nikita ""Ghost_RUS""",773640,2018-01-04,Colony On Mars,0,0,1,0,1,0,0,1,0,1
28847,Sacada,733530,2018-01-04,LOGistICAL: South Africa,0,0,1,0,1,0,0,0,0,1
28848,Laush Dmitriy Sergeevich,610660,2018-01-04,Russian Roads,0,0,0,0,1,0,1,1,0,0


procedemos a eliminar los valores nulos en la columna release_date para poder convertirla posteriormente en formato datetime

In [31]:
df['release_date'].replace("", np.nan, inplace=True)
df.dropna(subset=['release_date'], inplace=True)

verificamos si quedan nulos


In [32]:
num_nulos = df['release_date'].isnull().sum()

print(f"El DataFrame tiene {num_nulos} valores nulos en total.")


El DataFrame tiene 0 valores nulos en total.


aun debemos verificar que toda la columna posea las caracteristicas similares para la conversion

In [33]:

df['temp'] = pd.to_datetime(df['release_date'], errors='coerce')

# Los valores que no se pudieron convertir a datetime ahora son NaT (Not a Time)

# Verifica cuántos valores no se pudieron convertir
num_invalid = df['temp'].isnull().sum()

print(f"La columna 'release_date' tiene {num_invalid} valores con un formato inválido.")

# Elimina la columna temporal
df.drop('temp', axis=1, inplace=True)


La columna 'release_date' tiene 303 valores con un formato inválido.


pasamos a convertir la columna con el formato deseado, a posterior eliminamos las filas que son NaT

In [34]:
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')
df = df[df['release_date'].notna()]

In [35]:
df

,developer,id,release_date,title,Action,Adventure,Casual,Education,Indie,RPG,Racing,Simulation,Sports,Strategy
0,Kotoshiro,761140,2018-01-04,Lost Summoner Kitty,1,0,1,0,1,0,0,1,0,1
1,Secret Level SRL,643980,2018-01-04,Ironbound,0,0,0,0,1,1,0,0,0,1
2,Poolians.com,670290,2017-07-24,Real Pool 3D - Poolians,0,0,1,0,1,0,0,1,1,0
3,彼岸领域,767400,2017-12-07,弹炸人2222,1,1,1,0,0,0,0,0,0,0
4,Trickjump Games Ltd,772540,2018-01-04,Battle Royale Trainer,1,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28845,Bidoniera Games,745400,2018-01-04,Kebab it Up!,1,1,1,0,1,0,0,0,0,0
28846,"Nikita ""Ghost_RUS""",773640,2018-01-04,Colony On Mars,0,0,1,0,1,0,0,1,0,1
28847,Sacada,733530,2018-01-04,LOGistICAL: South Africa,0,0,1,0,1,0,0,0,0,1
28848,Laush Dmitriy Sergeevich,610660,2018-01-04,Russian Roads,0,0,0,0,1,0,1,1,0,0


Convertimos la columna id en datos enteros

In [36]:
df['id']=df['id'].astype(int)


C:\Users\Faccu E\AppData\Local\Temp\ipykernel_9580\163379862.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id']=df['id'].astype(int)


corroboramos los cambios

In [37]:
df.dtypes

developer               object
id                       int32
release_date    datetime64[ns]
title                   object
Action                   int64
Adventure                int64
Casual                   int64
Education                int64
Indie                    int64
RPG                      int64
Racing                   int64
Simulation               int64
Sports                   int64
Strategy                 int64
dtype: object

In [38]:
df

,developer,id,release_date,title,Action,Adventure,Casual,Education,Indie,RPG,Racing,Simulation,Sports,Strategy
0,Kotoshiro,761140,2018-01-04,Lost Summoner Kitty,1,0,1,0,1,0,0,1,0,1
1,Secret Level SRL,643980,2018-01-04,Ironbound,0,0,0,0,1,1,0,0,0,1
2,Poolians.com,670290,2017-07-24,Real Pool 3D - Poolians,0,0,1,0,1,0,0,1,1,0
3,彼岸领域,767400,2017-12-07,弹炸人2222,1,1,1,0,0,0,0,0,0,0
4,Trickjump Games Ltd,772540,2018-01-04,Battle Royale Trainer,1,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28845,Bidoniera Games,745400,2018-01-04,Kebab it Up!,1,1,1,0,1,0,0,0,0,0
28846,"Nikita ""Ghost_RUS""",773640,2018-01-04,Colony On Mars,0,0,1,0,1,0,0,1,0,1
28847,Sacada,733530,2018-01-04,LOGistICAL: South Africa,0,0,1,0,1,0,0,0,0,1
28848,Laush Dmitriy Sergeevich,610660,2018-01-04,Russian Roads,0,0,0,0,1,0,1,1,0,0


In [39]:
df['year'] = df['release_date'].dt.year

C:\Users\Faccu E\AppData\Local\Temp\ipykernel_9580\2122328270.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['release_date'].dt.year


In [40]:
df

,developer,id,release_date,title,Action,Adventure,Casual,Education,Indie,RPG,Racing,Simulation,Sports,Strategy,year
0,Kotoshiro,761140,2018-01-04,Lost Summoner Kitty,1,0,1,0,1,0,0,1,0,1,2018
1,Secret Level SRL,643980,2018-01-04,Ironbound,0,0,0,0,1,1,0,0,0,1,2018
2,Poolians.com,670290,2017-07-24,Real Pool 3D - Poolians,0,0,1,0,1,0,0,1,1,0,2017
3,彼岸领域,767400,2017-12-07,弹炸人2222,1,1,1,0,0,0,0,0,0,0,2017
4,Trickjump Games Ltd,772540,2018-01-04,Battle Royale Trainer,1,1,0,0,0,0,0,1,0,0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28845,Bidoniera Games,745400,2018-01-04,Kebab it Up!,1,1,1,0,1,0,0,0,0,0,2018
28846,"Nikita ""Ghost_RUS""",773640,2018-01-04,Colony On Mars,0,0,1,0,1,0,0,1,0,1,2018
28847,Sacada,733530,2018-01-04,LOGistICAL: South Africa,0,0,1,0,1,0,0,0,0,1,2018
28848,Laush Dmitriy Sergeevich,610660,2018-01-04,Russian Roads,0,0,0,0,1,0,1,1,0,0,2018


In [41]:
num_NaN = df['developer'].value_counts()['"NaN"']

In [42]:
num_NaN

169

In [43]:
df = df.loc[df['developer'] != '"NaN"']

In [44]:
df

,developer,id,release_date,title,Action,Adventure,Casual,Education,Indie,RPG,Racing,Simulation,Sports,Strategy,year
0,Kotoshiro,761140,2018-01-04,Lost Summoner Kitty,1,0,1,0,1,0,0,1,0,1,2018
1,Secret Level SRL,643980,2018-01-04,Ironbound,0,0,0,0,1,1,0,0,0,1,2018
2,Poolians.com,670290,2017-07-24,Real Pool 3D - Poolians,0,0,1,0,1,0,0,1,1,0,2017
3,彼岸领域,767400,2017-12-07,弹炸人2222,1,1,1,0,0,0,0,0,0,0,2017
4,Trickjump Games Ltd,772540,2018-01-04,Battle Royale Trainer,1,1,0,0,0,0,0,1,0,0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28845,Bidoniera Games,745400,2018-01-04,Kebab it Up!,1,1,1,0,1,0,0,0,0,0,2018
28846,"Nikita ""Ghost_RUS""",773640,2018-01-04,Colony On Mars,0,0,1,0,1,0,0,1,0,1,2018
28847,Sacada,733530,2018-01-04,LOGistICAL: South Africa,0,0,1,0,1,0,0,0,0,1,2018
28848,Laush Dmitriy Sergeevich,610660,2018-01-04,Russian Roads,0,0,0,0,1,0,1,1,0,0,2018


Creamos un PARQUET final con el ETL Realizado

In [45]:
df.to_csv('steamgames_final.csv', index=False)

In [47]:
df_recomendacion = df

In [48]:
print(df_recomendacion.index)

df_recomendacion

Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,    10,
       ...
       28840, 28841, 28842, 28843, 28844, 28845, 28846, 28847, 28848, 28849],
      dtype='int64', length=28547)


,developer,id,release_date,title,Action,Adventure,Casual,Education,Indie,RPG,Racing,Simulation,Sports,Strategy,year
0,Kotoshiro,761140,2018-01-04,Lost Summoner Kitty,1,0,1,0,1,0,0,1,0,1,2018
1,Secret Level SRL,643980,2018-01-04,Ironbound,0,0,0,0,1,1,0,0,0,1,2018
2,Poolians.com,670290,2017-07-24,Real Pool 3D - Poolians,0,0,1,0,1,0,0,1,1,0,2017
3,彼岸领域,767400,2017-12-07,弹炸人2222,1,1,1,0,0,0,0,0,0,0,2017
4,Trickjump Games Ltd,772540,2018-01-04,Battle Royale Trainer,1,1,0,0,0,0,0,1,0,0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28845,Bidoniera Games,745400,2018-01-04,Kebab it Up!,1,1,1,0,1,0,0,0,0,0,2018
28846,"Nikita ""Ghost_RUS""",773640,2018-01-04,Colony On Mars,0,0,1,0,1,0,0,1,0,1,2018
28847,Sacada,733530,2018-01-04,LOGistICAL: South Africa,0,0,1,0,1,0,0,0,0,1,2018
28848,Laush Dmitriy Sergeevich,610660,2018-01-04,Russian Roads,0,0,0,0,1,0,1,1,0,0,2018


Analisis de recomendacion de juegos Usando Cosine_similarity 

In [53]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
df.set_index("id", inplace=True)
columnas_numericas = df.select_dtypes(include=['number'])

scaler = StandardScaler()
columnas_numericas_normalizadas = pd.DataFrame(scaler.fit_transform(columnas_numericas), columns=columnas_numericas.columns, index=columnas_numericas.index)
similarity_matrix = cosine_similarity(columnas_numericas_normalizadas)
df_similarity = pd.DataFrame(similarity_matrix, index=columnas_numericas.index, columns=columnas_numericas.index)

In [54]:
df_similarity

id,761140,643980,670290,767400,772540,774276,774277,774278,768800,770380,...,761480,771810,767590,747320,769330,745400,773640,733530,610660,658870
id,,,,,,,,,,,,,,,,,,,,,
761140,1.000000,0.115534,0.245786,0.089971,0.179372,0.088464,0.088464,0.088464,0.245130,0.142104,...,-0.325306,-0.038242,0.216174,-0.268807,0.216174,0.283355,0.832013,0.562736,0.083764,0.190949
643980,0.115534,1.000000,-0.134454,-0.496158,-0.424702,-0.042628,-0.042628,-0.042628,-0.110779,0.199838,...,0.023213,-0.163115,0.022267,0.723255,0.022267,-0.283313,0.262784,0.434053,-0.039871,-0.014117
670290,0.245786,-0.134454,1.000000,-0.090446,-0.027115,0.916777,0.916777,0.916777,0.209438,-0.035798,...,-0.094563,-0.199332,0.258486,-0.072425,0.258486,0.030645,0.340840,0.134776,0.105957,0.254234
767400,0.089971,-0.496158,-0.090446,1.000000,0.465207,-0.293508,-0.293508,-0.293508,-0.073237,0.534409,...,0.145074,0.451376,0.138568,-0.422846,0.138568,0.775011,-0.150956,-0.030390,-0.276498,0.116555
772540,0.179372,-0.424702,-0.027115,0.465207,1.000000,0.069564,0.069564,0.069564,-0.009646,0.119287,...,0.183820,0.465548,-0.423944,-0.350612,-0.423944,0.268016,-0.041178,-0.486588,0.065969,-0.480157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745400,0.283355,-0.283313,0.030645,0.775011,0.268016,-0.162323,-0.162323,-0.162323,0.045690,0.722718,...,0.422699,0.708943,0.417648,-0.176851,0.417648,1.000000,0.051022,0.201773,-0.152751,0.394731
773640,0.832013,0.262784,0.340840,-0.150956,-0.041178,0.179913,0.179913,0.179913,0.334937,-0.055238,...,-0.148072,-0.315232,0.414646,-0.113119,0.414646,0.051022,1.000000,0.741629,0.170041,0.396603
733530,0.562736,0.434053,0.134776,-0.030390,-0.486588,-0.057710,-0.057710,-0.057710,0.143686,0.055517,...,0.002772,-0.201473,0.652690,0.021408,0.652690,0.201773,0.741629,1.000000,-0.054063,0.639326


funcion de consulta

In [54]:
def recommend_games(game_index, n=5):
    # Obtener la fila correspondiente al juego de referencia
    game_row = df_similarity.loc[game_index]

    # Ordenar la fila por similitud del coseno descendente y seleccionar los juegos más similares
    similar_games = game_row.sort_values(ascending=False).index[1:n+1]
    
    # Obtener los nombres de los juegos usando los ID
    similar_games_titles = df.loc[similar_games, 'title']

    return similar_games_titles

In [55]:
game_id = 730
game_name = df.loc[game_id, 'title']

In [56]:
recommended_games = recommend_games(game_id, n=5)
print(f"Juegos recomendados para '{game_id, game_name}': {recommended_games}")

Juegos recomendados para '(730, 'Counter-Strike: Global Offensive')': id
210040    Street Fighter X Tekken: Poison (Swap Costume)
115340                    Prototype 2 RADNET Access Pack
210044      Street Fighter X Tekken: Vega (Swap Costume)
210047      Street Fighter X Tekken: Juri (Swap Costume)
210054      Street Fighter X Tekken: Kuma (Swap Costume)
Name: title, dtype: object
